# WSI
## Raport z ćwiczenia laboratoryjnego nr 5
### Kacper Bugała, 303739 i Jan Kuc, 303753

### 1. Cel eksperymentów

Zadanie polega na implementacji perceptronu wielowarstwowego oraz wybrangeo algorytmu optymalizacji gradientowej z algorytem propagacji wstecznej. Następnie nalezy wytrenować stworzoną sieć do klasyfikacji odręcznie pisanych cyfr ze zbioru MNIST oraz sprawdzić efekty działania.

### 2. Opis eksperymentów i decyzje projektowe

Do implementacji perceptronu wielowarstwowego stworzono uniwersalną klasę **Network**, która korzysta z pomocniczej klasy **Layer**, dzięki czemu mozna zdefiniować dowolną liczbę warstw ukrytych sieci, podając równiez funkcję aktywacji oraz liczbę neuronów w warstwach ukrytych.

Ostatecznie zdecydowano się na trening oraz testowanie sieci składającej się z 2 warstw ukrytych aktywowanych funkcją *ReLu*. Funkcją aktywacji warstwy wyjściowej jest *softmax*. Funkcja ta, doskonale nadaje się do zadań klasyfikacji wieloklasowej i jest potrzebna do normalizacji wyników na końcu sieci. Dzięki niej na wyjściu otrzymujemy tak naprawdę głosowanie modelu na kazdą z mozliwych klas, a więc ocenione przez model prawdopodobieństwo przynalezności podanego przykładu wejściowego do danej klasy.

Jako algorytm opytmalizacji gradientowej wybrano SGD (Stochastic Gradient Descent). Trening modelu podzielony jest na określoną w parametrach wejśiowych liczbę **epok**, a w skład kazdej epoki wchodzą iteracje, których liczba jest zalezna od podanego w parametrach rozmiaru paczki, tzw. **batch size**.
Metoda podziału danych na losowe paczki w trakcie uczenia ma na celu przyspieszenie uczenia, poniewaz w takim przypadku w kazdej iteracji przez siec "przechodzi" tylko mały fragment danych wejściowych.

In [1]:
import sys
sys.path.append('../src/')
from network import Network
from data_reader import DataReader
from supporting_methods import ReLu, softmax, sigmoid
from sklearn.datasets import load_digits
import numpy as np

In [2]:
dataset = load_digits()

digits_data = dataset["data"]

digits_labels = dataset["target"]

In [3]:
data_reader = DataReader(data=digits_data, labels=digits_labels)

train_data, train_labels, test_data, test_labels, valid_data, valid_labels = data_reader.split_data()

In [4]:
input_size = train_data.shape[1]
input_size

64

In [5]:
output_size = train_labels.shape[1]
output_size

10

In [6]:
net = Network(hidden_count=2,
              hidden_size=10,
              input_size=input_size,
              output_size=output_size,
              hidden_act_fun=sigmoid,
              output_act_fun=softmax,
              learning_rate=0.2)

In [7]:
train_accs, train_losses, valid_accs, valid_losses = net.fit(epochs=1000,
        batch_size=20,
        train_data=train_data,
        train_labels=train_labels,
        valid_data=valid_data,
        valid_labels=valid_labels)

Epoch 1/1000
szsrj
(10, 1078)
Train data accuracy: 0.1011
train loss: 0.9155
szsrj
(10, 360)
Valid data accuracy: 0.1056
valid loss: 0.9128
---------------------------------------------------------
Epoch 2/1000
szsrj
(10, 1078)
Train data accuracy: 0.1011
train loss: 0.9299
szsrj
(10, 360)
Valid data accuracy: 0.1056
valid loss: 0.9251
---------------------------------------------------------
Epoch 3/1000
szsrj
(10, 1078)
Train data accuracy: 0.1020
train loss: 0.9504
szsrj
(10, 360)
Valid data accuracy: 0.1167
valid loss: 0.9440
---------------------------------------------------------
Epoch 4/1000
szsrj
(10, 1078)
Train data accuracy: 0.1020
train loss: 0.9753
szsrj
(10, 360)
Valid data accuracy: 0.1167
valid loss: 0.9676
---------------------------------------------------------
Epoch 5/1000
szsrj
(10, 1078)
Train data accuracy: 0.1020
train loss: 1.0061
szsrj
(10, 360)
Valid data accuracy: 0.1167
valid loss: 0.9969
---------------------------------------------------------
Epoch 6/10